# Environmental Variables

### Introduction

In this lesson, we'll discuss how to use Kubernetes to set environmental variables.  As we'll see, we can set environmental variables directly from our config files.  Let's get started.

### Environmental Variables

we can specify an environmental variable in our config file, and they will directly be available in the specified image.  For example, take a look at the bottom of the config file below.

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: backend-deployment
spec:
  replicas: 2
  selector:
    matchLabels:
      component: backend
  template:
    metadata:
      labels:
        component: backend
    spec:
      containers:
        - name: backend-api
          image: jek2141/apartments_api:v4
          ports:
          - containerPort: 5000
          env:
              name: DATABASE_URL
              value: http://postgres
```

And this means that if we now exec into one of our backend-api pods, we can access that environmental variable.

<img src="./db_url.png" width="40%">

Which we can always access from Python with a little help.

```python
import os
os.environ.get('DATABASE_URL')

# 'http://postgres'
```

### Using a ConfigMap

Now oftentime this configuration like a database url is something that we may need to reference throughout multiple deployments in our cluster.  Under our current mechanism, we would have to specify the keys and values for each deployment, and if we were up to update this database url, would need to make these updates in multiple locations.  

To avoid managing the same variable across multiple configuration files, kubernetes provides us with a separate object called a configMap.  Let's see how it works.  First we add the following yaml to a new file.

```yaml
# configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: database-config
  namespace: default
data:
  postgres_db_url: 'http://postgres'
```

And we can apply this file to create a new object with the following: `kubectl apply -f configmap.yaml`.  And from there we can check our work:

> <img src="./created_configmap.png" width="60%">

Next, we'll need to reference the values from our `database-config` in our deployment configuration file.  To accomplish this, we'll update the deployment config file like so:

```yaml
spec:
      containers:
        - name: backend-api
          image: jek2141/apartments_api:v4
          ports:
          - containerPort: 5000
          env:
              name: DATABASE_URL
              valueFrom:
                  configMapKeyRef: 
                      name: database-config
                      key: postgres_db_url
```

So notice that we are replaccing the `value` key, with the `valueFrom` key.  Then under the `configMapKeyRef` key, we need to specify the name and key of the configMap that we are referencing.  And if we apply our deployment file and `sh` into the shell, we can see that we still have working code.  

So the configMap is just a bit more indirect than using an environmental variable.  A configMap allows us to specify environmental variables that we can reference throughout our cluster.

### Secrets

There is another object that we can use to create an environmental variable and that is a secret.

```yaml
# secrets.yaml

apiVersion: v1
kind: Secret
metadata:
    name: postgres-credentials
stringData:
    postgres_password: "SECRET123"
```

Let's begin by creating a new secrets object.

`kubectl apply -f secrets.yaml`

And referencing data from a secrets object is similar to referencing it from a secrets object. 

```yaml
env: 
    - name: DATABASE_PASSWORD
            valueFrom:
              secretKeyRef:
                name: postgres-credentials
                key: postgres_password
```

So let's apply the changes and now see if we can access the `DATABASE_PASSWORD` as an environmental variable.

<img src="./db_password.png" width="60%">

Ok looks good.

So what was the point of using a secret?  Well notice that now if we look to describe our secret object, the information is obscured.

> <img src="./postgres_secret.png" width="60%">

So unlike with our configMap, we cannot see the value of secret simply by describing the object.

### Summary

In this lesson, we learned how to use Kubernetes to manage our environmental variables.
```yaml
# deployment.yaml
spec:
      containers:
        - name: backend-api
          image: jek2141/apartments_api:v4
          ports:
          - containerPort: 5000
          env:
              name: DATABASE_URL
              value: http://postgres
```

And we saw that if we want to manage these variables across multiple resources, we can first create a ConfigMap like we do below.

```yaml
# configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: database-config
  namespace: default
data:
  postgres_db_url: 'http://postgres'
```

And then reference the ConfigMap in the environmental variable with the ConfigMapKeyRef.

```yaml
# deployment.yaml
env:
  name: DATABASE_URL
  valueFrom:
      configMapKeyRef: 
          name: database-config
          key: postgres_db_url
```

So above, we do not directly store the database value, but rather reference it from the database-config.  Finally we saw that we can also use a secret if we wish to obscure the environmental variable value.

```yaml
# secrets.yaml

apiVersion: v1
kind: Secret
metadata:
    name: postgres-credentials
stringData:
    postgres_password: "SECRET123"
```

And can then again reference the environmental variable from the deployment config file, this time with the `secretKeyRef` value.

```yaml
- name: DATABASE_PASSWORD
    valueFrom:
      secretKeyRef:
        name: postgres-credentials
        key: postgres_password
```